In [102]:
import os
import shutil
import random
import pathlib
import pandas as pd

In [103]:
data = pd.read_csv("https://raw.githubusercontent.com/AnIsAsPe/ClassificadorCancerEsofago/master/Datos/ClasesImagenes.csv", usecols=[1,2])

In [104]:
data['class'] = data['class_number'] == 1

In [105]:
data.head()

,image_filename,class_number,class
0,im_4_0.png,0,False
1,im_21_0.png,0,False
2,im_9_0.png,0,False
3,im_8_0.png,0,False
4,im_15_0.png,0,False


In [106]:
data['class'].value_counts()

True     3594
False    1469
Name: class, dtype: int64

In [107]:
#Spliting and Copying into Train-Test Folders
target_dir = 'C:\\Users\\conte\\OneDrive\\Escritorio\\Colegio Bourbaki\\ML-AI-WA\\Perceptron\\ImagenesClasificadas\\'
image_path = 'C:\\Users\\conte\\OneDrive\\Escritorio\\Colegio Bourbaki\\ML-AI-WA\\Perceptron\\CarpetaImagenes\\'

count = 0
for image_split in [False, True]:
    labels = list(data[(data['class']==image_split)]['image_filename'])
    for label in labels:
        to_dir = pathlib.Path(str(target_dir) + str(image_split) + '/' + str(label))
        if not to_dir.is_dir():
            to_dir.parent.mkdir(parents=True, exist_ok=True)
        from_dir = pathlib.Path(image_path + label)
        shutil.copy2(from_dir, to_dir)
        count += 1

print(f'{count:.0f} images copied ')

5063 images copied 


In [97]:
# Setup data paths
data_path = 'C:\\Users\\conte\\OneDrive\\Escritorio\\Colegio Bourbaki\\ML-AI-WA\\Perceptron\\ImagenesClasificadas\\'

# Create function to split randomly in Train and Test
def train_test_split(image_path=data_path,
               data_splits=['Train', 'Test'], 
               target_classes = [False, True],
               split=0.3,
               seed=42):
    random.seed(seed)
    label_splits = {}
    
    # Get labels
    for data_split in data_splits:
        print(f"[INFO] Creating image split for: {data_split}...")
        image_paths = []
        for target in target_classes: 
            labels = list(data[(data['class']==target)]['image_filename'])
            sample = round((1-split)*len(labels))
            print(f"Getting random set of {sample} images for {data_split}-{target} ...")
            if data_split == 'Train':
                sampled_images = random.sample(labels, k=sample)
            elif data_split == 'Test':
                sampled_images = labels
            image_paths.append([pathlib.Path(str(image_path + str(target) + '/' + sample_image)) for sample_image in sampled_images])
            data.drop(data[data['image_filename'].isin(sampled_images)].index, inplace=True)
            print(len(data))        
        image_path_flat = [item for sublist in image_paths for item in sublist]
        label_splits[data_split] = image_path_flat
    
    print('\n','Finish splitting!')

    return label_splits

In [108]:
label_splits = train_test_split(split=0.3, seed=42)

[INFO] Creating image split for: Train...
Getting random set of 1028 images for Train-False ...
4035
Getting random set of 2516 images for Train-True ...
1519
[INFO] Creating image split for: Test...
Getting random set of 309 images for Test-False ...
1078
Getting random set of 755 images for Test-True ...
0

 Finish splitting!


In [109]:
label_splits['Train'][0:10]

[WindowsPath('C:/Users/conte/OneDrive/Escritorio/Colegio Bourbaki/ML-AI-WA/Perceptron/ImagenesClasificadas/False/im_277_31.png'),
 WindowsPath('C:/Users/conte/OneDrive/Escritorio/Colegio Bourbaki/ML-AI-WA/Perceptron/ImagenesClasificadas/False/im_98_4.png'),
 WindowsPath('C:/Users/conte/OneDrive/Escritorio/Colegio Bourbaki/ML-AI-WA/Perceptron/ImagenesClasificadas/False/im_46_1.png'),
 WindowsPath('C:/Users/conte/OneDrive/Escritorio/Colegio Bourbaki/ML-AI-WA/Perceptron/ImagenesClasificadas/False/im_173_6.png'),
 WindowsPath('C:/Users/conte/OneDrive/Escritorio/Colegio Bourbaki/ML-AI-WA/Perceptron/ImagenesClasificadas/False/im_137_6.png'),
 WindowsPath('C:/Users/conte/OneDrive/Escritorio/Colegio Bourbaki/ML-AI-WA/Perceptron/ImagenesClasificadas/False/im_56_6.png'),
 WindowsPath('C:/Users/conte/OneDrive/Escritorio/Colegio Bourbaki/ML-AI-WA/Perceptron/ImagenesClasificadas/False/im_42_5.png'),
 WindowsPath('C:/Users/conte/OneDrive/Escritorio/Colegio Bourbaki/ML-AI-WA/Perceptron/ImagenesClasif

In [110]:
from collections import Counter
counter1 = Counter(label_splits['Train'])
counter2 = Counter(label_splits['Test'])
common_elements = counter1 & counter2
len(common_elements)

0

In [111]:
# Create target directory path
target_dir_name = 'C:\\Users\\conte\\OneDrive\\Escritorio\\Colegio Bourbaki\\ML-AI-WA\\Perceptron\\Imagenes_Clasificadas_Random_Split'
print(f"Creating directory: '{target_dir_name}'")

# Setup the directories & Make the directories
target_dir = pathlib.Path(target_dir_name)
target_dir.mkdir(parents=True, exist_ok=True)


import shutil

count=0
for image_split in label_splits.keys():
    for image_path in label_splits[str(image_split)]:
        dest_dir = target_dir / image_split / image_path.parts[-2] / image_path.name
        if not dest_dir.parent.is_dir():
            dest_dir.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(image_path, dest_dir)
        count += 1
        

print(f'{count:.0f} images moved')

Creating directory: 'C:\Users\conte\OneDrive\Escritorio\Colegio Bourbaki\ML-AI-WA\Perceptron\Imagenes_Clasificadas_Random_Split'
5063 images moved


In [112]:
# for image_split in label_splits.keys():
#     delete_dir = target_dir / image_split
#     files_in_dir = os.listdir(delete_dir)     # get list of files in the directory
#     for file in files_in_dir:                  # loop to delete each file in folder
#         os.remove(f'{delete_dir}/{file}')     # delete file
#     os.rmdir(image_path.parent)